## Generate a Spice Subckt to Drive the PROBES Based on a Sizing JSON

In [21]:
import os
import json
from datetime import datetime

os.chdir("/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp")
print("Current working directory:", os.getcwd())

Current working directory: /Users/peterkinget/Library/Mobile Documents/com~apple~CloudDocs/BOX/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp


In [24]:
def generate_sizes_subckt(sizes_file, registers_file, output_json_file, output_spice_file):
    """
    Combines the generation of register settings and SPICE netlist into one function.

    Args:
        sizes_file (str): Path to the JSON file containing device sizes.
        registers_file (str): Path to the JSON file mapping devices to their registers.
        output_json_file (str): Path to the output JSON file where register settings will be saved.
        output_spice_file (str): Path to the output SPICE netlist file.
    """
    import json


    # Load sizes and registers
    with open(sizes_file, "r") as f:
        sizes = json.load(f)

    with open(registers_file, "r") as f:
        registers = json.load(f)

    # Initialize the output dictionary for register settings
    register_settings = {}

    # Generate register settings
    for device, bit_to_register in registers.items():
        size = sizes.get(device, [0])[0]  # Default size to 0 if device is not in sizes.json
        if not (0 <= size <= 31):  # Validate size is a 5-bit number
            print(f"Warning: Size {size} for device {device} is not a 5-bit number.")
            size = 0

        # Iterate through the registers and set their values based on the size
        for bit, register in sorted(bit_to_register.items(), key=lambda x: int(x[0])):
            register_settings[register] = 1 if int(bit) & size else 0

    # Sort the output dictionary by register number
    sorted_register_settings = dict(sorted(register_settings.items()))

    # Save the generated settings to a JSON file
    with open(output_json_file, "w") as f:
        json.dump(sorted_register_settings, f, indent=4)

    print(f"Register settings saved to {output_json_file}")

    # Generate SPICE netlist
    with open(output_spice_file, "w") as f:
        # Write the SPICE header
        f.write("simulator lang=spice\n\n")
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"* Generated PK_set_sizes netlist \n")
        f.write(f"*   for {sizes_file} \n")
        f.write(f"*   on {current_time}\n\n")

        # Write the subckt header
        subckt_header = '''.SUBCKT PK_set_sizes_2 PROBE<1889> PROBE<1890> PROBE<1891> PROBE<1892> PROBE<1893>
+  PROBE<1894> PROBE<1895> PROBE<1896> PROBE<1897> PROBE<1898> PROBE<1899>
+  PROBE<1900> PROBE<1901> PROBE<1902> PROBE<1903> PROBE<1904> PROBE<1905>
+  PROBE<1906> PROBE<1907> PROBE<1908> PROBE<1909> PROBE<1910> PROBE<1911>
+  PROBE<1912> PROBE<1913> PROBE<1914> PROBE<1915> PROBE<1916> PROBE<1917>
+  PROBE<1918> PROBE<1919> PROBE<1920> PROBE<1921> PROBE<1922> PROBE<1923>
+  PROBE<1924> PROBE<1925> PROBE<1926> PROBE<1927> PROBE<1928> PROBE<1929>
+  PROBE<1930> PROBE<1931> PROBE<1932> PROBE<1933> PROBE<1934> PROBE<1935>
+  PROBE<1936> PROBE<1937> PROBE<1938> PROBE<1939> PROBE<1940> PROBE<1941>
+  PROBE<1942> PROBE<1943> PROBE<1944> PROBE<1945> PROBE<1946> PROBE<1947>
+  PROBE<1948> PROBE<1949> PROBE<1950> PROBE<1951> PROBE<1952> PROBE<1953>
+  PROBE<1954> PROBE<1955> PROBE<1956> PROBE<1957> PROBE<1958> PROBE<1959>
+  PROBE<1960> PROBE<1961> PROBE<1962> PROBE<1963> PROBE<1964> PROBE<1965>
+  PROBE<1966> PROBE<1967> PROBE<1968> PROBE<1969> PROBE<1970> PROBE<1971>
+  PROBE<1972> PROBE<1973> PROBE<1974> PROBE<1975> PROBE<1976> PROBE<1977>
+  PROBE<1978> PROBE<1979> PROBE<1980> PROBE<1981> PROBE<1982> PROBE<1983>
+  PROBE<1984> PROBE<1985> PROBE<1986> PROBE<1987> PROBE<1988> PROBE<1989>
+  PROBE<1990> PROBE<1991> PROBE<1992> PROBE<1993> PROBE<1994> PROBE<1995>
+  PROBE<1996> PROBE<1997> PROBE<1998> PROBE<1999> PROBE<2000> PROBE<2001>
+  PROBE<2002> PROBE<2003> PROBE<2004> PROBE<2005> PROBE<2006> PROBE<2007>
+  PROBE<2008> VDD VSS
'''
        f.write(subckt_header + "\n")

        # Iterate through each device, sorted by device name
        for device in sorted(registers.keys()):
            f.write(f"* Device: {device} Size: {sizes.get(device, [0])[0]} \n")  # Add a comment for the device
            bit_to_register = registers[device]
            for bit, register in sorted(bit_to_register.items(), key=lambda x: int(x[0])):
                register_value = sorted_register_settings.get(register, 0)
                if register_value == 1:
                    f.write(f"V_{device}_{register} PROBE<{register}> VDD 0\n")
                else:
                    f.write(f"V_{device}_{register} PROBE<{register}> VSS 0\n")
            f.write("\n")  # Add a blank line after each device group

        # Write the SPICE footer
        f.write(".ENDS\n")

    print(f"SPICE netlist saved to {output_spice_file}")

In [25]:
sizes_file = "../../examples/INV_string_5_growing_sizes.json"
registers_file = "device_name_to_sizing_registers.json"
output_json_file = "generated_register_settings.json"
output_spice_file = "generated_set_sizes_PROBE.cir"
generate_sizes_subckt(sizes_file, registers_file, output_json_file, output_spice_file)

Register settings saved to generated_register_settings.json
SPICE netlist saved to generated_set_sizes_PROBE.cir


## OLDER Functions

In [6]:
# sizes to dictionary for registers

# File paths
sizes_file = "../../examples/INV_string_5_growing_sizes.json"
registers_file = "device_name_to_sizing_registers.json"
output_file = "generated_register_settings.json"

# Load sizes and registers
with open(sizes_file, "r") as f:
    sizes = json.load(f)

with open(registers_file, "r") as f:
    registers = json.load(f)

# Initialize the output dictionary
register_settings = {}

# Generate register settings
for device, bit_to_register in registers.items():
    size = sizes.get(device, [0])[0]  # Default size to 0 if device is not in sizes.json
    if not (0 <= size <= 31):  # Validate size is a 5-bit number
        print(f"Warning: Size {size} for device {device} is not a 5-bit number.")
        size = 0

    # Iterate through the registers and set their values based on the size
    for bit, register in sorted(bit_to_register.items(), key=lambda x: int(x[0])):
        register_settings[register] = 1 if int(bit) & size else 0

# Sort the output dictionary by register number
sorted_register_settings = dict(sorted(register_settings.items()))

# Save the generated settings to a JSON file
with open(output_file, "w") as f:
    json.dump(sorted_register_settings, f, indent=4)

print(f"Register settings saved to {output_file}")


NameError: name 'json' is not defined

In [ ]:
def generate_register_settings(sizes_file, registers_file, output_file):
    """
    Generates a dictionary of register settings based on device sizes and saves it to a JSON file.

    Args:
        sizes_file (str): Path to the JSON file containing device sizes.
        registers_file (str): Path to the JSON file mapping devices to their registers.
        output_file (str): Path to the output JSON file where register settings will be saved.
    """
    import json

    # Load sizes and registers
    with open(sizes_file, "r") as f:
        sizes = json.load(f)

    with open(registers_file, "r") as f:
        registers = json.load(f)

    # Initialize the output dictionary
    register_settings = {}

    # Generate register settings
    for device, bit_to_register in registers.items():
        size = sizes.get(device, [0])[0]  # Default size to 0 if device is not in sizes.json
        if not (0 <= size <= 31):  # Validate size is a 5-bit number
            print(f"Warning: Size {size} for device {device} is not a 5-bit number.")
            size = 0

        # Iterate through the registers and set their values based on the size
        for bit, register in sorted(bit_to_register.items(), key=lambda x: int(x[0])):
            register_settings[register] = 1 if int(bit) & size else 0

    # Sort the output dictionary by register number
    sorted_register_settings = dict(sorted(register_settings.items()))

    # Save the generated settings to a JSON file
    with open(output_file, "w") as f:
        json.dump(sorted_register_settings, f, indent=4)

    print(f"Register settings saved to {output_file}")

In [ ]:
sizes_file = "../../examples/INV_string_5_growing_sizes.json"
registers_file = "device_name_to_sizing_registers.json"
output_file = "generated_set_sizes_PROBE.json"
generate_register_settings(sizes_file, registers_file, output_file)

Register settings saved to generated_register_settings.json


In [ ]:
def generate_register_spice_netlist(register_settings_file, registers_file, output_spice_file):
    """
    Generates a SPICE netlist based on register settings.
    Registers set to 1 are connected to VDD, and those set to 0 are connected to VSS.
    Comments are added per device level, grouped by 5 bits, sorted by device name.

    Args:
        register_settings_file (str): Path to the generated register settings JSON file.
        registers_file (str): Path to the device-to-registers mapping JSON file.
        output_spice_file (str): Path to the output SPICE netlist file.
    """
    # Load the register settings
    with open(register_settings_file, "r") as f:
        register_settings = json.load(f)

    # Load the device-to-registers mapping
    with open(registers_file, "r") as f:
        registers = json.load(f)

    # Open the output SPICE file for writing
    with open(output_spice_file, "w") as f:
        # Write the SPICE header
        f.write("simulator lang=spice\n\n")
        f.write("* Generated SPICE netlist for register settings\n\n")

        # Write the subckt header
        subckt_header = '''.SUBCKT PK_set_sizes PROBE<1889> PROBE<1890> PROBE<1891> PROBE<1892> PROBE<1893>
+  PROBE<1894> PROBE<1895> PROBE<1896> PROBE<1897> PROBE<1898> PROBE<1899>
+  PROBE<1900> PROBE<1901> PROBE<1902> PROBE<1903> PROBE<1904> PROBE<1905>
+  PROBE<1906> PROBE<1907> PROBE<1908> PROBE<1909> PROBE<1910> PROBE<1911>
+  PROBE<1912> PROBE<1913> PROBE<1914> PROBE<1915> PROBE<1916> PROBE<1917>
+  PROBE<1918> PROBE<1919> PROBE<1920> PROBE<1921> PROBE<1922> PROBE<1923>
+  PROBE<1924> PROBE<1925> PROBE<1926> PROBE<1927> PROBE<1928> PROBE<1929>
+  PROBE<1930> PROBE<1931> PROBE<1932> PROBE<1933> PROBE<1934> PROBE<1935>
+  PROBE<1936> PROBE<1937> PROBE<1938> PROBE<1939> PROBE<1940> PROBE<1941>
+  PROBE<1942> PROBE<1943> PROBE<1944> PROBE<1945> PROBE<1946> PROBE<1947>
+  PROBE<1948> PROBE<1949> PROBE<1950> PROBE<1951> PROBE<1952> PROBE<1953>
+  PROBE<1954> PROBE<1955> PROBE<1956> PROBE<1957> PROBE<1958> PROBE<1959>
+  PROBE<1960> PROBE<1961> PROBE<1962> PROBE<1963> PROBE<1964> PROBE<1965>
+  PROBE<1966> PROBE<1967> PROBE<1968> PROBE<1969> PROBE<1970> PROBE<1971>
+  PROBE<1972> PROBE<1973> PROBE<1974> PROBE<1975> PROBE<1976> PROBE<1977>
+  PROBE<1978> PROBE<1979> PROBE<1980> PROBE<1981> PROBE<1982> PROBE<1983>
+  PROBE<1984> PROBE<1985> PROBE<1986> PROBE<1987> PROBE<1988> PROBE<1989>
+  PROBE<1990> PROBE<1991> PROBE<1992> PROBE<1993> PROBE<1994> PROBE<1995>
+  PROBE<1996> PROBE<1997> PROBE<1998> PROBE<1999> PROBE<2000> PROBE<2001>
+  PROBE<2002> PROBE<2003> PROBE<2004> PROBE<2005> PROBE<2006> PROBE<2007>
+  PROBE<2008> VDD VSS
'''
        f.write(subckt_header + "\n")

        # Iterate through each device, sorted by device name
        for device in sorted(registers.keys()):
            f.write(f"* Device: {device}\n")  # Add a comment for the device
            bit_to_register = registers[device]
            for bit, register in sorted(bit_to_register.items(), key=lambda x: int(x[0])):
                register_value = register_settings.get(str(register), 0)
                if register_value == 1:
                    f.write(f"V_{device}_{register} PROBE<{register}> VDD 0\n")
                else:
                    f.write(f"V_{device}_{register} PROBE<{register}> VSS 0\n")
            f.write("\n")  # Add a blank line after each device group

        # Write the SPICE footer
        f.write(".ENDS\n")

    print(f"SPICE netlist saved to {output_spice_file}")

In [ ]:
def print_device_sizes_sorted(register_settings_file, registers_file):
    """
    Reads the generated register settings and prints out the sizes for the various devices,
    sorted by device name.

    Args:
        register_settings_file (str): Path to the generated register settings JSON file.
        registers_file (str): Path to the device-to-registers mapping JSON file.
    """
    # Load the register settings
    with open(register_settings_file, "r") as f:
        register_settings = json.load(f)

    # Load the device-to-registers mapping
    with open(registers_file, "r") as f:
        registers = json.load(f)

    # Initialize a dictionary to store the sizes for each device
    device_sizes = {}

    # Calculate the size for each device
    for device, bit_to_register in registers.items():
        size = 0
        for bit, register in bit_to_register.items():
            if register_settings.get(str(register), 0) == 1:  # Check if the register is set to 1
                size += int(bit)
        device_sizes[device] = size

    # Sort the devices by name and print the sizes
    for device in sorted(device_sizes.keys()):
        print(f"{device}: {device_sizes[device]}")

# Example usage
register_settings_file = "generated_register_settings.json"
registers_file = "device_name_to_sizing_registers.json"
print_device_sizes_sorted(register_settings_file, registers_file)

CC_N: 1
CC_P: 1
DCC1_N_L: 5
DCC1_N_R: 5
DCC1_P_L: 5
DCC1_P_R: 5
DCC2_N_L: 7
DCC2_N_R: 7
DCC2_P_L: 7
DCC2_P_R: 7
DCC3_N_L: 9
DCC3_N_R: 9
DCC3_P_L: 9
DCC3_P_R: 9
DCC4_N_L: 11
DCC4_N_R: 11
DCC4_P_L: 11
DCC4_P_R: 11
DINV1_L: 13
DINV1_R: 13
DINV2_L: 15
DINV2_R: 15
OTA_N: 1
OTA_P: 1


In [ ]:
# Example usage
register_settings_file = "generated_register_settings.json"
registers_file = "device_name_to_sizing_registers.json"
output_spice_file = "generated_register_spice_netlist.cir"
generate_register_spice_netlist(register_settings_file, registers_file, output_spice_file)